In [ ]:
import os
os.environ["WANDB_DISABLED"]="true"

In [ ]:
! pip install datasets transformers sacrebleu torch sentencepiece transformers[sentencepiece]

     |████████████████████████████████| 270 kB 5.2 MB/s 
     |████████████████████████████████| 2.9 MB 38.9 MB/s 
     |████████████████████████████████| 90 kB 9.0 MB/s 
     |████████████████████████████████| 1.2 MB 44.8 MB/s 
     |████████████████████████████████| 243 kB 45.9 MB/s 
     |████████████████████████████████| 1.3 MB 33.0 MB/s 
     |████████████████████████████████| 125 kB 47.7 MB/s 
     |████████████████████████████████| 56 kB 4.7 MB/s 
     |████████████████████████████████| 895 kB 29.8 MB/s 
     |████████████████████████████████| 636 kB 44.9 MB/s 
     |████████████████████████████████| 3.3 MB 33.3 MB/s 
     |████████████████████████████████| 160 kB 44.0 MB/s 
     |████████████████████████████████| 271 kB 33.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import transformers
print(transformers.__version__)

4.11.3


In [ ]:
model_checkpoint = "Helsinki-NLP/opus-mt-en-ro"

In [ ]:
from datasets import load_dataset, load_metric
raw_datasets = load_dataset("wmt16", "ro-en")
metric = load_metric("sacrebleu")

Downloading:   0%|          | 0.00/1.41k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.69k [00:00<?, ?B/s]

  0%|          | 0/4 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/225M [00:00<?, ?B/s]

...

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset wmt16 downloaded and prepared to /root/.cache/huggingface/datasets/wmt16/ro-en/1.0.0/0d9fb3e814712c785176ad8cdb9f465fbe6479000ee6546725db30ad8a8b5f8a. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/2.37k [00:00<?, ?B/s]

In [ ]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 610320
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 1999
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 1999
    })
})

In [ ]:
raw_datasets["train"][0]

{'translation': {'en': 'Membership of Parliament: see Minutes',
  'ro': 'Componenţa Parlamentului: a se vedea procesul-verbal'}}

In [ ]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML
def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))
show_random_elements(raw_datasets["train"])

,translation
0,"{'en': 'I think that it is a slight understatement, Mr President, when you say that it is an honour for you to take up the position to which we have elected you today.', 'ro': 'Cred că este o uşoară dovadă de modestie, dle Preşedinte, când spuneţi că este o onoare pentru dvs. să preluaţi poziţia în care aţi fost ales astăzi.'}"
1,"{'en': 'What advice would you give to the Slovak Prime Minister Fico, who now faces a 'Sophie's Choice'?', 'ro': 'Ce sfaturi i-ați da Prim Ministrului slovac Fico, care se confruntă acum cu o ""Alegere a Sofiei""?'}"
2,"{'en': 'Cesare Battisti is a convicted murderer in Italy, and the stance taken by the Brazilian authorities to protect him from the reach of EU justice is not only unacceptable but also condemnable and deplorable in the strongest possible terms and we, and any other authority and any other body in the EU, have every right to say so.', 'ro': 'Cesare Battisti este un criminal condamnat în Italia, iar poziţia adoptată de autorităţile braziliene de a-l proteja de braţul justiţiei din UE nu este doar inacceptabilă, ci şi regretabilă şi condamnabilă cu cea mai mare fermitate, iar noi, la fel ca orice altă autoritate sau organism al UE, avem tot dreptul să o spunem.'}"
3,"{'en': 'Oprisan has said he would like to see someone new at the party helm -- for example, former Foreign Minister Mircea Geoana, former Transportation Minister Miron Mitrea, former Defence Minister Ioan Mircea Pascu or former Justice Minister Cristian Diaconescu.', 'ro': 'Oprişan a afirmat că ar dori pe cineva nou la conducerea partidului -- de exemplu, fostul ministru de externe Mircea Geoană, fostul ministru al transporturilor Miron Mitrea, fostul ministru al apărării Ioan Mircea Paşcu sau fostul ministru al justiţiei Cristian Diaconescu.'}"
4,"{'en': 'Their businesses need access to clear legislation and their consumers need to have access to a wider range of goods at more competitive prices.', 'ro': 'Afacerile lor au nevoie de acces la o legislație clară, iar consumatorii lor trebuie să aibă acces la o gamă mai largă de bunuri la prețuri mai competitive.'}"


In [ ]:
metric

Metric(name: "sacrebleu", features: {'predictions': Value(dtype='string', id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id='references')}, usage: """
Produces BLEU scores along with its sufficient statistics
from a source against one or more references.

Args:
    predictions: The system stream (a sequence of segments).
    references: A list of one or more reference streams (each a sequence of segments).
    smooth_method: The smoothing method to use. (Default: 'exp').
    smooth_value: The smoothing value. Only valid for 'floor' and 'add-k'. (Defaults: floor: 0.1, add-k: 1).
    tokenize: Tokenization method to use for BLEU. If not provided, defaults to 'zh' for Chinese, 'ja-mecab' for
        Japanese and '13a' (mteval) otherwise.
    lowercase: Lowercase the data. If True, enables case-insensitivity. (Default: False).
    force: Insist that your tokenized input is actually detokenized.

Returns:
    'score': BLEU score,
    'counts'

In [ ]:
fake_preds = ["hello there", "general kenobi"]
fake_labels = [["hello there"], ["general kenobi"]]
metric.compute(predictions=fake_preds, references=fake_labels)

{'bp': 1.0,
 'counts': [4, 2, 0, 0],
 'precisions': [100.0, 100.0, 0.0, 0.0],
 'ref_len': 4,
 'score': 0.0,
 'sys_len': 4,
 'totals': [4, 2, 0, 0]}

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/770k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.33M [00:00<?, ?B/s]

In [ ]:
tokenizer(["Hello, this one sentence!", "This is another sentence."])

{'input_ids': [[125, 778, 3, 63, 141, 9191, 23, 0], [187, 32, 716, 9191, 2, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}

In [ ]:
with tokenizer.as_target_tokenizer():
    print(tokenizer(["Hello, this one sentence!", "This is another sentence."]))

{'input_ids': [[10334, 1204, 3, 15, 8915, 27, 452, 59, 29579, 581, 23, 0], [235, 1705, 11, 32, 8, 1205, 5305, 59, 29579, 581, 2, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


In [ ]:
prefix = ""
max_input_length = 128
max_target_length = 128
source_lang = "en"
target_lang = "ro"
def preprocess_function(examples):
    inputs = [prefix + ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)
    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
preprocess_function(raw_datasets['train'][:2])

{'input_ids': [[393, 4462, 14, 1137, 53, 216, 28636, 0], [24385, 14, 28636, 14, 4646, 4622, 53, 216, 28636, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[42140, 494, 1750, 53, 8, 59, 903, 3543, 9, 15202, 0], [36199, 6612, 9, 15202, 122, 568, 35788, 21549, 53, 8, 59, 903, 3543, 9, 15202, 0]]}

In [ ]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

  0%|          | 0/611 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

Fine Tuning the Model

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/287M [00:00<?, ?B/s]

In [ ]:
batch_size = 16
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-{source_lang}-to-{target_lang}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True
)

Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
import numpy as np
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `MarianMTModel.forward` and have been ignored: translation.
***** Running training *****
  Num examples = 610320
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 38145


Epoch,Training Loss,Validation Loss


Saving model checkpoint to opus-mt-en-ro-finetuned-en-to-ro/checkpoint-500
Configuration saved in opus-mt-en-ro-finetuned-en-to-ro/checkpoint-500/config.json
Model weights saved in opus-mt-en-ro-finetuned-en-to-ro/checkpoint-500/pytorch_model.bin
tokenizer config file saved in opus-mt-en-ro-finetuned-en-to-ro/checkpoint-500/tokenizer_config.json
Special tokens file saved in opus-mt-en-ro-finetuned-en-to-ro/checkpoint-500/special_tokens_map.json
Saving model checkpoint to opus-mt-en-ro-finetuned-en-to-ro/checkpoint-1000
Configuration saved in opus-mt-en-ro-finetuned-en-to-ro/checkpoint-1000/config.json
Model weights saved in opus-mt-en-ro-finetuned-en-to-ro/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in opus-mt-en-ro-finetuned-en-to-ro/checkpoint-1000/tokenizer_config.json
Special tokens file saved in opus-mt-en-ro-finetuned-en-to-ro/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to opus-mt-en-ro-finetuned-en-to-ro/checkpoint-1500
Configuration saved i

In [ ]:
import os
for dirname, _, filenames in os.walk('opus-mt-en-ro-finetuned-en-to-ro'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

opus-mt-en-ro-finetuned-en-to-ro/checkpoint-37000/vocab.json
opus-mt-en-ro-finetuned-en-to-ro/checkpoint-37000/rng_state.pth
opus-mt-en-ro-finetuned-en-to-ro/checkpoint-37000/config.json
opus-mt-en-ro-finetuned-en-to-ro/checkpoint-37000/tokenizer_config.json
opus-mt-en-ro-finetuned-en-to-ro/checkpoint-37000/trainer_state.json
opus-mt-en-ro-finetuned-en-to-ro/checkpoint-37000/scheduler.pt
opus-mt-en-ro-finetuned-en-to-ro/checkpoint-37000/special_tokens_map.json
opus-mt-en-ro-finetuned-en-to-ro/checkpoint-37000/pytorch_model.bin
opus-mt-en-ro-finetuned-en-to-ro/checkpoint-37000/target.spm
opus-mt-en-ro-finetuned-en-to-ro/checkpoint-37000/source.spm
opus-mt-en-ro-finetuned-en-to-ro/checkpoint-37000/training_args.bin
opus-mt-en-ro-finetuned-en-to-ro/checkpoint-37000/optimizer.pt
opus-mt-en-ro-finetuned-en-to-ro/runs/Sep30_03-49-58_a84265f68a25/events.out.tfevents.1632973810.a84265f68a25.217.0
opus-mt-en-ro-finetuned-en-to-ro/runs/Sep30_03-49-58_a84265f68a25/1632973810.56194/events.out.tfev

In [ ]:
from transformers import MarianMTModel, MarianTokenizer
src_text = ['My name is Sarah and I live in London']

model_name = 'opus-mt-en-ro-finetuned-en-to-ro/checkpoint-38000'
tokenizer = MarianTokenizer.from_pretrained(model_name)
print(tokenizer.supported_language_codes)


Didn't find file opus-mt-en-ro-finetuned-en-to-ro/checkpoint-38000/added_tokens.json. We won't load it.
Didn't find file opus-mt-en-ro-finetuned-en-to-ro/checkpoint-38000/tokenizer.json. We won't load it.
loading file opus-mt-en-ro-finetuned-en-to-ro/checkpoint-38000/source.spm
loading file opus-mt-en-ro-finetuned-en-to-ro/checkpoint-38000/target.spm
loading file opus-mt-en-ro-finetuned-en-to-ro/checkpoint-38000/vocab.json
loading file opus-mt-en-ro-finetuned-en-to-ro/checkpoint-38000/tokenizer_config.json
loading file None
loading file opus-mt-en-ro-finetuned-en-to-ro/checkpoint-38000/special_tokens_map.json
loading file None


[]


In [ ]:
model = MarianMTModel.from_pretrained(model_name)
translated = model.generate(**tokenizer(src_text, return_tensors="pt", padding=True))
[tokenizer.decode(t, skip_special_tokens=True) for t in translated]

loading configuration file opus-mt-en-ro-finetuned-en-to-ro/checkpoint-38000/config.json
Model config MarianConfig {
  "_name_or_path": "Helsinki-NLP/opus-mt-en-ro",
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "swish",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "MarianMTModel"
  ],
  "attention_dropout": 0.0,
  "bad_words_ids": [
    [
      59542
    ]
  ],
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 512,
  "decoder_attention_heads": 8,
  "decoder_ffn_dim": 2048,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 59542,
  "dropout": 0.1,
  "encoder_attention_heads": 8,
  "encoder_ffn_dim": 2048,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 0,
  "forced_eos_token_id": 0,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "label2id": {
    

['Numele meu este Sarah şi locuiesc la Londra']